In [1]:
!pip install -U ray

     |████████████████████████████████| 22.9 MB 11.8 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 9.2 MB/s s eta 0:00:01
     |████████████████████████████████| 274 kB 118.7 MB/s eta 0:00:01
     |████████████████████████████████| 126 kB 123.8 MB/s eta 0:00:01
     |████████████████████████████████| 2.9 MB 101.5 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 20.3 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 17.7 MB/s  eta 0:00:01
     |████████████████████████████████| 45 kB 8.3 MB/s  eta 0:00:01
     |████████████████████████████████| 1.2 MB 92.0 MB/s eta 0:00:01
     |████████████████████████████████| 201 kB 110.6 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 79.5 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 18.3 MB/s eta 0:00:01
     |████████████████████████████████| 91 kB 3.5 MB/s s eta 0:00:01
     |████████████████████████████████| 257 kB 113.8 MB/s eta 0:00:01
     |█████████████████████████

  Created wheel for gpustat: filename=gpustat-0.6.0-py3-none-any.whl size=12617 sha256=17ed5d266eac5f65705e9ca3ca440fe0fde54fa88e9ff705d9ea12c2d6d19bc4
  Stored in directory: /home/ec2-user/.cache/pip/wheels/50/da/35/fe2cfb3bc47822299f5e124a599d56f00b30ec0b328db16b9f
  Created wheel for nvidia-ml-py3: filename=nvidia_ml_py3-7.352.0-py3-none-any.whl size=19189 sha256=886140e3450a2771c092b0e81bb681368b5b85d2fb62442d47c5761165eeb6de
  Stored in directory: /home/ec2-user/.cache/pip/wheels/7f/26/a3/33f2079871e2bebb3f53a2b21c3ec64129b8efdd18a6263a52
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3161 sha256=6ff9f05bb3a82c3cd0ede90c92438e6e66728f05b114536118387aa08ce60dce
  Stored in directory: /home/ec2-user/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
  Created wheel for contextvars: filename=contextvars-2.4-py3-none-any.whl size=7664 sha256=32b8837c2cc2be4cd1482aeaa5209d1d5e6b27c54849253b25f31e8a2cceb517
  Stored in directory: /

In [2]:
!pip install tb-nightly

     |████████████████████████████████| 10.6 MB 14.4 MB/s eta 0:00:01
     |████████████████████████████████| 94 kB 8.0 MB/s s eta 0:00:01
     |████████████████████████████████| 127 kB 106.6 MB/s eta 0:00:01
     |████████████████████████████████| 779 kB 85.9 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 109.2 MB/s eta 0:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [3]:
!pip install future

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install tabulate

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.


In [1]:
import os
import time
import copy

from collections import OrderedDict

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

from torch.utils.tensorboard import SummaryWriter

import matplotlib.pyplot as plt
import numpy as np

from functools import partial
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

print(torch.cuda.is_available())

False


In [2]:
SEED = 42

In [3]:
"""
Create train, valid, test iterators for CIFAR-10 [1].
Easily extended to MNIST, CIFAR-100 and Imagenet.
[1]: https://discuss.pytorch.org/t/feedback-on-pytorch-for-kaggle-competitions/2252/4
"""

def plot_images(images, cls_true, cls_pred=None):
    """
    Adapted from https://github.com/Hvass-Labs/TensorFlow-Tutorials/
    """
    fig, axes = plt.subplots(3, 3)

    for i, ax in enumerate(axes.flat):
        # plot img
        ax.imshow(images[i, :, :, :], interpolation='spline16')

        # show true & predicted classes
        cls_true_name = label_names[cls_true[i]]
        if cls_pred is None:
            xlabel = "{0} ({1})".format(cls_true_name, cls_true[i])
        else:
            cls_pred_name = label_names[cls_pred[i]]
            xlabel = "True: {0}\nPred: {1}".format(
                cls_true_name, cls_pred_name
            )
        ax.set_xlabel(xlabel)
        ax.set_xticks([])
        ax.set_yticks([])

    plt.show()

def get_train_valid_loader(data_dir,
                           batch_size,
                           augment,
                           random_seed,
                           valid_size=0.1,
                           shuffle=True,
                           show_sample=False,
                           num_workers=4,
                           pin_memory=False):
    """
    Utility function for loading and returning train and valid
    multi-process iterators over the CIFAR-10 dataset. A sample
    9x9 grid of the images can be optionally displayed.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - augment: whether to apply the data augmentation scheme
      mentioned in the paper. Only applied on the train split.
    - random_seed: fix seed for reproducibility.
    - valid_size: percentage split of the training set used for
      the validation set. Should be a float in the range [0, 1].
    - shuffle: whether to shuffle the train/validation indices.
    - show_sample: plot 9x9 sample grid of the dataset.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - train_loader: training set iterator.
    - valid_loader: validation set iterator.
    """
    error_msg = "[!] valid_size should be in the range [0, 1]."
    assert ((valid_size >= 0) and (valid_size <= 1)), error_msg

    normalize = transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2023, 0.1994, 0.2010],
    )

    # define transforms
    valid_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
    ])
    if augment:
        train_transform = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize,
        ])
    else:
        train_transform = transforms.Compose([
            transforms.ToTensor(),
            normalize,
        ])

    # load the dataset
    train_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=train_transform,
    )

    valid_dataset = datasets.CIFAR10(
        root=data_dir, train=True,
        download=True, transform=valid_transform,
    )

    num_train = len(train_dataset)
    indices = list(range(num_train))
    split = int(np.floor(valid_size * num_train))

    if shuffle:
        np.random.seed(random_seed)
        np.random.shuffle(indices)

    train_idx, valid_idx = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_idx)
    valid_sampler = SubsetRandomSampler(valid_idx)

    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, sampler=train_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=batch_size, sampler=valid_sampler,
        num_workers=num_workers, pin_memory=pin_memory,
    )

    # visualize some images
    if show_sample:
        sample_loader = torch.utils.data.DataLoader(
            train_dataset, batch_size=9, shuffle=shuffle,
            num_workers=num_workers, pin_memory=pin_memory,
        )
        data_iter = iter(sample_loader)
        images, labels = data_iter.next()
        X = images.numpy().transpose([0, 2, 3, 1])
        plot_images(X, labels)

    loaders = {"Train":train_loader, "Valid":valid_loader} 
    dataset_sizes = {"Train": len(train_dataset), "Valid": len(train_dataset) * valid_size}
    return loaders, dataset_sizes


def get_test_loader(data_dir,
                    batch_size,
                    shuffle=True,
                    num_workers=4,
                    pin_memory=False):
    """
    Utility function for loading and returning a multi-process
    test iterator over the CIFAR-10 dataset.
    If using CUDA, num_workers should be set to 1 and pin_memory to True.
    Params
    ------
    - data_dir: path directory to the dataset.
    - batch_size: how many samples per batch to load.
    - shuffle: whether to shuffle the dataset after every epoch.
    - num_workers: number of subprocesses to use when loading the dataset.
    - pin_memory: whether to copy tensors into CUDA pinned memory. Set it to
      True if using GPU.
    Returns
    -------
    - data_loader: test set iterator.
    """
    normalize = transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )

    # define transform
    transform = transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ])

    dataset = datasets.CIFAR10(
        root=data_dir, train=False,
        download=True, transform=transform,
    )

    data_loader = torch.utils.data.DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle,
        num_workers=num_workers, pin_memory=pin_memory,
    )
    loader = {"Test":data_loader}
    dataset_sizes = {"Test": len(dataset)}
    return loader, dataset_sizes

In [4]:
# Training the model
def train_model(model, device, dataloaders, dataset_sizes, criterion, optimizer, tb_logger, num_epochs, scheduler=None):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['Train', 'Valid']:
            if phase == 'Train':
                if scheduler:
                    scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                # DEVICE --> where the computation is happening
                # Torch.dtypes --> cuda tensors, cpu tensors

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'Train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'Train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            #Log with Tensorboard
            if phase=='Train':
                tb_logger.add_scalar('train_Loss',float(epoch_loss), epoch+1)
                tb_logger.add_scalar('train_Accuracy', float(epoch_acc), epoch+1)
            elif phase=='Valid':
                tb_logger.add_scalar('valid_Loss',float(epoch_loss), epoch+1)
                tb_logger.add_scalar('valid_Accuracy', float(epoch_acc), epoch+1)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'Valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, epoch_loss, best_acc

def test_model(model, device, dataloaders, dataset_sizes, criterion):

    since = time.time()
    
    with torch.no_grad():
        for phase in ['Test']:
            model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Test Acc: {:4f}'.format(epoch_acc))

    return epoch_acc

In [5]:
class Net(nn.Module):
    def __init__(self, c1=32, c2=64, c3=64, l1=64):
        super(Net, self).__init__()

        self.conv_block = nn.Sequential(OrderedDict([
          ('conv1', nn.Conv2d(3, c1, 3)),
          ('relu1', nn.ReLU()),
          ('pool1', nn.MaxPool2d(2,2)),
          ('conv2', nn.Conv2d(c1,c2,3)),
          ('relu2', nn.ReLU()),
          ('pool2', nn.MaxPool2d(2,2)),
          ('conv3', nn.Conv2d(c2,64,3)),
          ('relu3', nn.ReLU()),
          ('flatten', nn.Flatten())
        ]))
        self.fc_block = nn.Sequential(OrderedDict([
          ('fc1', nn.Linear(1024, l1)),
          ('relu1', nn.ReLU()),
          ('fc2', nn.Linear(l1, 10))
        ]))

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc_block(x)
        return x

In [6]:
def train(config, checkpoint_dir = None, data_dir="./data"):

    net = Net(config["c1"], config["c2"], config["c3"], config["l1"])

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)

    criterion = nn.CrossEntropyLoss()
    scheduler = None
    if config["optim"] == "SGD":
        optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
    elif config["optim"] == "Adam":
        optimizer = optim.Adam(net.parameters(), lr=config["lr"])

    # The `checkpoint_dir` parameter gets passed by Ray Tune when a checkpoint
    # should be restored.
    if checkpoint_dir:
        checkpoint = os.path.join(checkpoint_dir, "checkpoint")
        model_state, optimizer_state = torch.load(checkpoint)
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    augment=False
    valid_size=0.2
    shuffle=True
    trainloaders, train_dataset_sizes = get_train_valid_loader(data_dir=data_dir, batch_size=config["batchsize"],
                       augment=augment, random_seed=SEED, valid_size=valid_size,
                       shuffle=shuffle, show_sample=False, num_workers=8, pin_memory=True)
    testloader, test_dataset_sizes = get_test_loader(data_dir=data_dir, batch_size=64, shuffle=False,
                             num_workers=8, pin_memory=True)
    
    tb_logger = SummaryWriter(os.path.join(LOG_DIR,"test_exp2"))
    num_epochs = 10

    model, val_loss, val_acc = train_model(net, device, trainloaders, train_dataset_sizes, criterion, optimizer, tb_logger, num_epochs)
    
    test_acc = test_model(net, device, testloader, test_dataset_sizes, criterion)

    tune.report(loss=val_loss, accuracy=val_acc)
    tb_logger.close()

In [7]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=0):
    data_dir = os.path.abspath("./data")
    # load_data(data_dir)  # Download data for all trials before starting the run
    config = {
        "c1": tune.choice([16, 32, 64]),
        "c2": tune.choice([32, 64, 128]),
        "c3": tune.choice([64, 128, 256]),        
        "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
        "lr": tune.loguniform(1e-4, 1e-1),
        "batchsize": tune.choice([16, 32, 64,]),
        "optim": tune.choice(["Adam"]),
    }
    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train, data_dir=data_dir),
        resources_per_trial={"cpu": 1, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = Net(best_trial.config["c1"], best_trial.config["c2"],
                            best_trial.config["c3"], best_trial.config["l1"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    checkpoint_path = os.path.join(best_trial.checkpoint.value, "checkpoint")

    model_state, optimizer_state = torch.load(checkpoint_path)
    best_trained_model.load_state_dict(model_state)

    testloader, test_dataset_sizes = get_test_loader(data_dir=data_dir, batch_size=64, shuffle=False,
                             num_workers=8, pin_memory=True)
    test_acc = test_model(best_trained_model, device, testloader, test_dataset_sizes, criterion)
    print("Best trial test set accuracy: {}".format(test_acc))

In [8]:
LOG_DIR = 'runs'
os.makedirs(LOG_DIR, exist_ok=True)

In [10]:
import torchvision.models as models


In [9]:
main()

2020-10-10 12:20:49,471	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265
2020-10-10 12:20:50,591	WARNING experiment.py:256 -- No name detected on trainable. Using DEFAULT.
2020-10-10 12:20:50,593	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.
2020-10-10 12:20:50,627	INFO logger.py:201 -- pip install 'ray[tune]' to see TensorBoard files.
2020-10-10 12:20:50,628	WARNING logger.py:343 -- Could not instantiate TBXLogger: No module named 'tensorboardX'.


== Status ==
Memory usage on this node: 1.4/15.1 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1/8 CPUs, 0/0 GPUs, 0.0/8.64 GiB heap, 0.0/2.98 GiB objects
Result logdir: /home/ec2-user/ray_results/DEFAULT
Number of trials: 10 (9 PENDING, 1 RUNNING)
+---------------------+----------+-------+-------------+------+------+------+------+-------------+---------+
| Trial name          | status   | loc   |   batchsize |   c1 |   c2 |   c3 |   l1 |          lr | optim   |
|---------------------+----------+-------+-------------+------+------+------+------+-------------+---------|
| DEFAULT_08cf9_00000 | RUNNING  |       |           8 |   32 |   32 |  256 |   64 | 0.00050722  | SGD     |
| DEFAULT_08cf9_00001 | PENDING  |       |           4 |   64 |  128 |  128 |    4 | 0.00241188  | SGD     |
| DEFAULT_08cf9_00002 | PENDING  |       |          16 |   16 |  128 |  256 |  256 | 0.000698859 | SGD     |

(pid=10257) Train Loss: 0.8469 Acc: 0.5021
(pid=10257) Valid Loss: 1.0328 Acc: 0.6360
(pid=10257) Epoch 6/10
(pid=10257) ----------
(pid=10251) Train Loss: 0.7928 Acc: 0.5187
(pid=10256) Train Loss: 1.4462 Acc: 0.1980
(pid=10247) Train Loss: 0.7122 Acc: 0.5524
(pid=10251) Valid Loss: 0.9525 Acc: 0.6617
(pid=10251) Epoch 4/10
(pid=10251) ----------
(pid=10248) Train Loss: 0.7878 Acc: 0.5147
(pid=10256) Valid Loss: 1.8269 Acc: 0.2595
(pid=10256) Epoch 4/10
(pid=10256) ----------
(pid=10247) Valid Loss: 0.9591 Acc: 0.6664
(pid=10247) Epoch 6/10
(pid=10247) ----------
(pid=10248) Valid Loss: 1.0669 Acc: 0.6169
(pid=10248) Epoch 7/10
(pid=10248) ----------
(pid=10249) Train Loss: 1.0951 Acc: 0.3946
(pid=10246) Train Loss: 0.7770 Acc: 0.5262
(pid=10249) Valid Loss: 1.3073 Acc: 0.5334
(pid=10249) Epoch 3/10
(pid=10249) ----------
(pid=10257) Train Loss: 0.7727 Acc: 0.5271
(pid=10246) Valid Loss: 1.0354 Acc: 0.6367
(pid=10246) Epoch 6/10
(pid=10246) ----------
(pid=10257) Valid Loss: 0.9830 Ac

(pid=10248) 2020-10-10 12:33:06,925	ERROR function_runner.py:233 -- Runner Thread raised error.
(pid=10248) Traceback (most recent call last):
(pid=10248)   File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ray/tune/function_runner.py", line 227, in run
(pid=10248)     self._entrypoint()
(pid=10248)   File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ray/tune/function_runner.py", line 290, in entrypoint
(pid=10248)     self._status_reporter.get_checkpoint())
(pid=10248)   File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ray/tune/function_runner.py", line 499, in _trainable_func
(pid=10248)     output = train_func(config, checkpoint_dir=checkpoint_dir)
(pid=10248)   File "<ipython-input-6-4caa60c98e21>", line 39, in train
(pid=10248)   File "<ipython-input-4-fcba8162403f>", line 76, in train_model
(pid=10248) NameError: name 'best_model' is not defined
(pid=10248) Exception in thread Thread-2:
(pid=10248) Tra

(pid=10248) Valid Loss: 0.9780 Acc: 0.6682
(pid=10248) Training complete in 12m 12s
(pid=10248) Best val Acc: 0.668200
== Status ==
Memory usage on this node: 7.1/15.1 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 7/8 CPUs, 0/0 GPUs, 0.0/8.64 GiB heap, 0.0/2.98 GiB objects
Result logdir: /home/ec2-user/ray_results/DEFAULT
Number of trials: 10 (1 ERROR, 2 PENDING, 7 RUNNING)
+---------------------+----------+-------+-------------+------+------+------+------+-------------+---------+
| Trial name          | status   | loc   |   batchsize |   c1 |   c2 |   c3 |   l1 |          lr | optim   |
|---------------------+----------+-------+-------------+------+------+------+------+-------------+---------|
| DEFAULT_08cf9_00000 | RUNNING  |       |           8 |   32 |   32 |  256 |   64 | 0.00050722  | SGD     |
| DEFAULT_08cf9_00001 | RUNNING  |       |           4 |   64 |  128 |  128 |    4 | 0.00

...

(pid=17254) 2020-10-10 12:51:13,563	ERROR function_runner.py:233 -- Runner Thread raised error.
(pid=17254) Traceback (most recent call last):
(pid=17254)   File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ray/tune/function_runner.py", line 227, in run
(pid=17254)     self._entrypoint()
(pid=17254)   File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ray/tune/function_runner.py", line 290, in entrypoint
(pid=17254)     self._status_reporter.get_checkpoint())
(pid=17254)   File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ray/tune/function_runner.py", line 499, in _trainable_func
(pid=17254)     output = train_func(config, checkpoint_dir=checkpoint_dir)
(pid=17254)   File "<ipython-input-6-4caa60c98e21>", line 39, in train
(pid=17254)   File "<ipython-input-4-fcba8162403f>", line 76, in train_model
(pid=17254) NameError: name 'best_model' is not defined
(pid=17254) Exception in thread Thread-2:
(pid=17254) Tra

(pid=17254) Valid Loss: 2.3054 Acc: 0.1030
(pid=17254) Training complete in 16m 59s
(pid=17254) Best val Acc: 0.104000
== Status ==
Memory usage on this node: 2.5/15.1 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 1/8 CPUs, 0/0 GPUs, 0.0/8.64 GiB heap, 0.0/2.98 GiB objects
Result logdir: /home/ec2-user/ray_results/DEFAULT
Number of trials: 10 (9 ERROR, 1 RUNNING)
+---------------------+----------+-------+-------------+------+------+------+------+-------------+---------+
| Trial name          | status   | loc   |   batchsize |   c1 |   c2 |   c3 |   l1 |          lr | optim   |
|---------------------+----------+-------+-------------+------+------+------+------+-------------+---------|
| DEFAULT_08cf9_00000 | ERROR    |       |           8 |   32 |   32 |  256 |   64 | 0.00050722  | SGD     |
| DEFAULT_08cf9_00001 | ERROR    |       |           4 |   64 |  128 |  128 |    4 | 0.00241188  | S

(pid=10255) 2020-10-10 12:58:23,632	ERROR function_runner.py:233 -- Runner Thread raised error.
(pid=10255) Traceback (most recent call last):
(pid=10255)   File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ray/tune/function_runner.py", line 227, in run
(pid=10255)     self._entrypoint()
(pid=10255)   File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ray/tune/function_runner.py", line 290, in entrypoint
(pid=10255)     self._status_reporter.get_checkpoint())
(pid=10255)   File "/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ray/tune/function_runner.py", line 499, in _trainable_func
(pid=10255)     output = train_func(config, checkpoint_dir=checkpoint_dir)
(pid=10255)   File "<ipython-input-6-4caa60c98e21>", line 39, in train
(pid=10255)   File "<ipython-input-4-fcba8162403f>", line 76, in train_model
(pid=10255) NameError: name 'best_model' is not defined
(pid=10255) Exception in thread Thread-2:
(pid=10255) Tra

(pid=10255) Valid Loss: 0.9255 Acc: 0.7190
(pid=10255) Training complete in 37m 28s
(pid=10255) Best val Acc: 0.719000
== Status ==
Memory usage on this node: 1.7/15.1 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 0/8 CPUs, 0/0 GPUs, 0.0/8.64 GiB heap, 0.0/2.98 GiB objects
Result logdir: /home/ec2-user/ray_results/DEFAULT
Number of trials: 10 (10 ERROR)
+---------------------+----------+-------+-------------+------+------+------+------+-------------+---------+
| Trial name          | status   | loc   |   batchsize |   c1 |   c2 |   c3 |   l1 |          lr | optim   |
|---------------------+----------+-------+-------------+------+------+------+------+-------------+---------|
| DEFAULT_08cf9_00000 | ERROR    |       |           8 |   32 |   32 |  256 |   64 | 0.00050722  | SGD     |
| DEFAULT_08cf9_00001 | ERROR    |       |           4 |   64 |  128 |  128 |    4 | 0.00241188  | SGD     |
|

TuneError: ('Trials did not complete', [DEFAULT_08cf9_00000, DEFAULT_08cf9_00001, DEFAULT_08cf9_00002, DEFAULT_08cf9_00003, DEFAULT_08cf9_00004, DEFAULT_08cf9_00005, DEFAULT_08cf9_00006, DEFAULT_08cf9_00007, DEFAULT_08cf9_00008, DEFAULT_08cf9_00009])